## Scikit-Learn 랜덤 포레스트 개발, 훈련, 최적화 및 배포

이 노트북은 SageMaker Studio의 Python 3 (Data Science 3.0) 커널이나 기존 SageMaker 노트북 인스턴스의 *conda_python3 에서 잘 작동합니다

이 노트북에서는 Amazon SageMaker를 사용하여 인기 있는 ML 프레임워크인 Scikit-Learn을 기반으로 랜덤 포레스트 모델을 개발, 훈련, 튜닝 및 배포하는 방법을 보여줍니다.
이 예제는 Scikit-Learn에서 제공하는 캘리포니아 주택 데이터셋을 사용합니다 - 자세한 내용은 여기에서 확인할 수 있습니다.
코드를 이해하기 위해 다음 자료들을 참고하시면 도움이 될 것입니다:

* The guide on [Using Scikit-Learn with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/using_sklearn.html)
* The API doc for [Scikit-Learn classes in the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/sagemaker.sklearn.html)
* The [SageMaker reference for Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#client) (The general AWS SDK for Python, including low-level bindings for SageMaker as well as many other AWS services)


## 라이브러리 및 환경 설정
위에서 언급한 SageMaker 커널(올바른 커널에서 실행 중인지 확인하세요!)에는 이미 이 노트북에 필요한 모든 라이브러리가 포함되어 있습니다 - 따라서 현재 AWS 리전과 대상 Amazon S3 버킷과 같은 기본 구성을 설정하고 의존성을 가져오는 것부터 시작할 수 있습니다:
- [AWS Region](https://aws.amazon.com/about-aws/global-infrastructure/) 
- [target Amazon S3 bucket](https://aws.amazon.com/s3/):


In [14]:
# Python Built-Ins:
import os

# External Dependencies:
import boto3  # General-purpose AWS SDK for Python
import numpy as np  # Tools for working with numeric arrays
import pandas as pd  # Tools for warking with data tables (dataframes)
import sagemaker  # High-level SDK for Amazon SageMaker in particular
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print(f"Using bucket {bucket}")

Using bucket sagemaker-us-east-1-057716757052


## 데이터 준비

다음으로, SKLearn에서 원시 예제 데이터셋을 로드하고 훈련 작업에서 사용할 형식으로 준비하겠습니다: 
훈련용과 검증/테스트용으로 구분된 별도의 CSV 파일입니다.
```
데이터셋 개요

1990년 캘리포니아 인구조사 데이터 기반
캘리포니아 각 지역의 주택 가격 중앙값과 관련 특성들을 포함
총 20,640개의 데이터 포인트 포함

특성(Features)

MedInc: 해당 지역의 중간 소득
HouseAge: 주택의 평균 연령
AveRooms: 평균 방 개수
AveBedrms: 평균 침실 수
Population: 해당 지역 인구
AveOccup: 평균 가구원 수
Latitude: 위도
Longitude: 경도

타겟 변수(Target)
주택 가격의 중앙값(단위: 100,000달러)
```

In [16]:
data = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX["target"] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX["target"] = y_test

trainX.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,4.2143,37.0,5.288235,0.973529,860.0,2.529412,33.81,-118.12,2.285
1,5.3468,42.0,6.364322,1.087940,957.0,2.404523,37.16,-121.98,2.799
2,3.9191,36.0,6.110063,1.059748,711.0,2.235849,38.45,-122.69,1.830
3,6.3703,32.0,6.000000,0.990196,1159.0,2.272549,34.16,-118.41,4.658
4,2.3684,17.0,4.795858,1.035503,706.0,2.088757,38.57,-121.33,1.500


In [17]:
# create directories
os.makedirs("data/train", exist_ok=True)
os.makedirs("data/test", exist_ok=True)
os.makedirs("src", exist_ok=True)
os.makedirs("model", exist_ok=True)

# save data as csv
trainX.to_csv("data/train/train.csv")
testX.to_csv("data/test/test.csv")

## 훈련 스크립트 생성
SageMaker Scikit-Learn [프레임워크 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-docker-containers-scikit-learn-spark.html)는 기본 런타임을 제공하며, 사용자는 실행할 실제 훈련 단계를 스크립트 파일(또는 requirements.txt를 포함한 여러 파일이 있는 폴더)로 지정합니다.
아래 코드는 노트북에서 [src/main.py](src/main.py) 파일을 초기화합니다. 런처나 파일 메뉴에서도 Python 스크립트와 다른 파일들을 생성할 수 있습니다.
이 예제에서는 동일한 파일이 훈련 시(스크립트로 실행)와 추론 시([모듈](https://docs.python.org/3/tutorial/modules.html)로 가져옴)에 사용됩니다 - 따라서 아래에서는:

기본 동작을 재정의하기 위한 특정 추론 함수들(예: model_fn())을 정의하고,
훈련 진입점을 if __name__ == '__main__' [가드 절](https://docs.python.org/3/library/__main__.html)로 감싸서 모듈이 스크립트로 실행될 때만 실행되도록 합니다.

자세한 가이드는 Scikit-Learn 훈련 스크립트 준비하기(훈련용)와 SageMaker Scikit-Learn 모델 서버(추론용) 문서에서 찾을 수 있습니다.
- [Preparing a Scikit-Learn training script](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#prepare-a-scikit-learn-training-script) (for training)
- [SageMaker Scikit-Learn model server](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#sagemaker-scikit-learn-model-server) (for inference).



In [18]:
%%writefile src/main.py
# Python Built-Ins:
import argparse
import os

# External Dependencies:
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# ---- INFERENCE FUNCTIONS ----
def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model


if __name__ == "__main__":
    # ---- TRAINING ENTRY POINT ----
    
    # Arguments like data location and hyper-parameters are passed from SageMaker to your script
    # via command line arguments and/or environment variables. You can use Python's built-in
    # argparse module to parse them:
    print("Parsing training arguments")
    parser = argparse.ArgumentParser()

    # RandomForest hyperparameters
    parser.add_argument("--n_estimators", type=int, default=10)
    parser.add_argument("--min_samples_leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model_dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train_file", type=str, default="train.csv")
    parser.add_argument("--test_file", type=str, default="test.csv")
    parser.add_argument("--features", type=str)  # explicitly name which features to use
    parser.add_argument("--target_variable", type=str)  # name the column to be used as target

    args, _ = parser.parse_known_args()

    # -- DATA PREPARATION --
    # Load the data from the local folder(s) SageMaker pointed us to:
    print("Reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("Building training and testing datasets")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target_variable]
    y_test = test_df[args.target_variable]

    # -- MODEL TRAINING --
    print("Training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        min_samples_leaf=args.min_samples_leaf,
        n_jobs=-1)

    model.fit(X_train, y_train)

    # -- MODEL EVALUATION --
    print("Testing model")
    abs_err = np.abs(model.predict(X_test) - y_test)
    # Output metrics to the console (in this case, percentile absolute errors):
    for q in [10, 50, 90]:
        print(f"AE-at-{q}th-percentile: {np.percentile(a=abs_err, q=q)}")

    # -- SAVE THE MODEL --
    # ...To the specific folder SageMaker pointed us to:
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print(f"model saved at {path}")


Overwriting src/main.py


## 로컬 훈련
구성이 명령줄 인수로 이루어지므로, SageMaker 훈련 작업에 업로드하기 전에 훈련 스크립트를 로컬에서 테스트할 수 있습니다.

⚠️ 참고: 이는 작은 샘플 데이터셋으로 스크립트를 빠르게 기능 테스트하는 데 좋습니다... 하지만 스크립트가 기능적으로 작동한다고 확신하면, 실험을 재현 가능하고 추적 가능한 SageMaker 훈련 작업으로 옮기는 것이 좋습니다. 노트북 커널의 라이브러리가 나중에 훈련 작업을 위해 구성하는 컨테이너 이미지와 정확히 일치하지 않을 수 있다는 점에 유의하세요.

In [19]:
!python src/main.py \
    --n_estimators 100 \
    --min_samples_leaf 3 \
    --model_dir model/ \
    --train data/train \
    --test data/test \
    --features 'MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude' \
    --target_variable target

Parsing training arguments
Reading data
Building training and testing datasets
Training model
Testing model
AE-at-10th-percentile: 0.032603306410256425
AE-at-50th-percentile: 0.20534242569930095
AE-at-90th-percentile: 0.7784335334858474
model saved at model/model.joblib


## SageMaker 훈련
스크립트를 훈련 작업에서 실행하려면 먼저 SageMaker가 접근할 수 있는 곳에 데이터를 업로드해야 합니다: 일반적으로 이는 [Amazon S3](https://aws.amazon.com/s3/)가 될 것입니다.

### 데이터 입력 "채널" 생성하기 (S3에 복사)

SageMaker의 여러 데이터 "채널"의 수와 이름 지정은 사용자에게 달려 있습니다: 정확히 2개일 필요가 없으며, "train"과 "test"로 불릴 필요도 없습니다.



In [20]:
train_data_s3uri = sess.upload_data(
    path="data/train/train.csv",  # Local source
    bucket=bucket,
    key_prefix="sm101/sklearn-cali/train",  # Destination path in S3 bucket
)

test_data_s3uri = sess.upload_data(
    path="data/test/test.csv",  # Local source
    bucket=bucket,
    key_prefix="sm101/sklearn-cali/test",  # Destination path in S3 bucket
)

print("Train set URI:", train_data_s3uri)
print("Test set URI:", test_data_s3uri)

Train set URI: s3://sagemaker-us-east-1-057716757052/sm101/sklearn-cali/train/train.csv
Test set URI: s3://sagemaker-us-east-1-057716757052/sm101/sklearn-cali/test/test.csv


## Python SDK로 훈련 작업 시작하기
데이터를 업로드하고 스크립트를 준비했다면, SageMaker 훈련 작업을 구성할 준비가 된 것입니다:



In [21]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
    entry_point="main.py",
    source_dir="src",  # To upload the whole folder - or instead set entry_point="src/main.py"
    role=sagemaker.get_execution_role(),  # Use same IAM role as notebook is currently using
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.0-1",
    base_job_name="rf-scikit",
    metric_definitions=[
        # SageMaker can extract metrics from your console logs via Regular Expressions:
        {"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"},
    ],
    hyperparameters={
        "n_estimators": 100,
        "min_samples_leaf": 3,
        "features": "MedInc HouseAge AveRooms AveBedrms Population AveOccup Latitude Longitude",
        "target_variable": "target",
        # SageMaker data channels are always folders. Even if you point to a particular object
        # S3URI, you'll need to either: Properly support loading folder inputs in your script; or
        # use extra configuration parameters to identify specific filename(s):
        "train_file": "train.csv",
        "test_file": "test.csv",
    },
    # Optional settings to run with SageMaker Managed Spot:
    max_run=20*60,  # Maximum allowed active runtime (in seconds)
    use_spot_instances=True,  # Use spot instances to reduce cost
    max_wait=30*60,  # Maximum clock time (including spot delays)
)

In [22]:
sklearn_estimator.fit({"train": train_data_s3uri, "test": test_data_s3uri}, wait=True)

[12/17/24 01:33:32] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=740084;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=34041;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[12/17/24 01:33:33] INFO     Creating training-job with name: rf-scikit-2024-12-17-01-33-32-945     ]8;id=176104;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=595754;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2024-12-17 01:33:34 Starting - Starting the training job...
2024-12-17 01:33:48 Starting - Preparing the instances for training...
2024-12-17 01:34:18 Downloading - Downloading input data...
2024-12-17 01:35:04 Downloading - Downloading the training image......
2024-12-17 01:35:55 Training - Training image download completed. Training in progress.2024-12-17 01:35:59,446 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-12-17 01:35:59,450 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-17 01:35:59,453 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-17 01:35:59,470 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-12-17 01:35:59,771 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-17 01:35:59,776 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)


우리가 실행한 훈련 작업은 매우 작은 데이터셋으로 인해 매우 "가벼운" 작업입니다. 따라서 노트북 인스턴스에서 로컬로 실행하는 것이 SageMaker에서 실행하는 것보다 실행 시간이 더 빠릅니다. SageMaker는 훈련 인프라를 프로비저닝해야 하기 때문에 작업 실행에 더 오랜 시간이 걸립니다. 이 예제 훈련 작업은 리소스 집약적이지 않기 때문에 인프라 프로비저닝 프로세스가 훈련 작업 자체보다 더 많은 오버헤드를 추가합니다.

실제 상황에서는 데이터셋이 크기 때문에 SageMaker에서 실행하면 실행 프로세스를 상당히 가속화할 수 있으며, 이 대화형 노트북 환경을 적당하게 유지하고 더 강력한 훈련 작업 리소스를 온디맨드로 가동함으로써 비용을 최적화하는 데 도움이 됩니다.

이 훈련 작업은 *노트북 자체에서 실행되지 않았습니다*. [AWS Console for SageMaker - Training Jobs 탭](https://console.aws.amazon.com/sagemaker/home?#/jobs)과 [SageMaker Studio Experiments and Trials UI](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-view-compare.html)에서 기록을 볼 수 있습니다.

> ℹ️ **팁:** 노트북 커널이 다시 시작되거나 추정기 상태가 다른 이유로 손실된 경우 훈련 작업을 **다시 실행할 필요가 없습니다**. 이름으로 이전 훈련 작업에 *연결*하기만 하면 됩니다. 예를 들면:
>
> ```python
> estimator = SKLearn.attach("rf-scikit-2025-01-01-00-00-00-000")
> ```


## 실시간 엔드포인트에 배포하기

### Python SDK로 배포하기

훈련된 `Estimator`를 SageMaker 엔드포인트에 한 줄의 코드로 실시간 추론을 위해 배포할 수 있습니다. `Estimator.deploy(...)`를 사용하면 암시적으로 SageMaker [모델](https://console.aws.amazon.com/sagemaker/home?#/models), [엔드포인트 구성](https://console.aws.amazon.com/sagemaker/home?#/endpointConfig), [엔드포인트](https://console.aws.amazon.com/sagemaker/home?#/endpoints)을 생성합니다.

그러나 더 세밀한 제어를 원할 경우 SageMaker Python SDK를 통해 `Model` 객체를 생성할 수 있습니다. 이 경우 훈련 작업에 의해 Amazon S3에 생성된 `model.tar.gz`를 참조합니다. 이를 통해 다음과 같은 작업을 수행할 수 있습니다:

- 훈련과 추론 사이에 사용되는 환경 변수나 Python 파일을 수정합니다.
- SageMaker 외부에서 훈련된 모델을 Amazon S3에 호환 가능한 `model.tar.gz`로 패키징하여 가져옵니다.

여기서는 더 긴 경로를 시연하겠습니다:

In [23]:
sklearn_estimator.latest_training_job.wait(logs="None")  # Check the job is finished

# describe() here is equivalent to low-level boto3 SageMaker describe_training_job
job_desc = sklearn_estimator.latest_training_job.describe()
model_s3uri = job_desc["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact saved at:", model_s3uri)


2024-12-17 01:36:28 Starting - Preparing the instances for training
2024-12-17 01:36:28 Downloading - Downloading the training image
2024-12-17 01:36:28 Training - Training image download completed. Training in progress.
2024-12-17 01:36:28 Uploading - Uploading generated training model
2024-12-17 01:36:28 Completed - Training job completed
Model artifact saved at: s3://sagemaker-us-east-1-057716757052/rf-scikit-2024-12-17-01-33-32-945/output/model.tar.gz


In [24]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=model_s3uri,
    framework_version="1.0-1",
    py_version="py3",
    role=sagemaker.get_execution_role(),
    entry_point="src/main.py",
)

In [25]:
predictor = model.deploy(
    instance_type="ml.c5.large",
    initial_instance_count=1,
)

[12/17/24 01:47:20] INFO     Creating model with name:                                              ]8;id=450583;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=646582;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             sagemaker-scikit-learn-2024-12-17-01-47-20-181                                        

                    INFO     Creating endpoint-config with name                                     ]8;id=830389;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=148888;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-scikit-learn-2024-12-17-01-47-20-862                                        

[12/17/24 01:47:21] INFO     Creating endpoint with name                                            ]8;id=720745;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=832413;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\
                             sagemaker-scikit-learn-2024-12-17-01-47-20-862                                        

-------!

### 실시간 추론

SageMaker Python SDK의 [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) 클래스는 엔드포인트를 감싸는 Python 래퍼를 제공하며, 요청 및 응답의 (구성 가능한) 직렬화/역직렬화도 처리합니다.

SageMaker Python SDK를 사용할 수 없는 클라이언트(예: 비 Python 클라이언트 또는 어떤 이유로든 PyPI의 [sagemaker](https://pypi.org/project/sagemaker/) 패키지를 설치할 수 없는 Python 환경)의 경우: 일반 AWS SDK를 사용하여 하위 수준 [SageMaker InvokeEndpoint API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html)를 호출할 수 있습니다.


In [26]:
# the SKLearnPredictor does the serialization from pandas for us
print(predictor.predict(testX[data.feature_names]))

[0.49221684 0.8189902  4.86482757 ... 1.29014649 2.97195452 4.0161205 ]


### 엔드포인트 삭제하기

훈련 작업 인프라는 작업이 시작되면 즉시 시작되고 작업이 중지되면 종료되지만, 엔드포인트는 우리가 종료할 때까지 활성 상태로 유지됩니다. 지속적인 비용을 방지하기 위해 사용하지 않는 엔드포인트를 삭제하십시오:

In [27]:
predictor.delete_endpoint(delete_endpoint_config=True)

[12/17/24 01:52:09] INFO     Deleting endpoint configuration with name:                             ]8;id=968119;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=226730;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\
                             sagemaker-scikit-learn-2024-12-17-01-47-20-862                                        

                    INFO     Deleting endpoint with name:                                           ]8;id=227638;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=86315;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\
                             sagemaker-scikit-learn-2024-12-17-01-47-20-862                                        

## 결론

이 노트북에서 우리는 다음의 예제를 보았습니다:

- 구성 가능한 매개변수와 출력 정확도 측정항목을 가진 SageMaker 훈련 작업으로 Scikit-Learn 기반 모델 훈련 스크립트를 실행합니다.
- 훈련된 모델을 실시간 추론 API에 배포합니다.

SageMaker는 불필요한 코드 없이 모델 서빙 스택을 처리했습니다: 필요한 경우 기본 동작을 사용자 정의하기 위해 [오버라이드 함수](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#sagemaker-scikit-learn-model-server) (예: `input_fn` 및 `model_fn`)만 정의하면 됩니다. 훈련 시간에 우리의 스크립트는 SageMaker를 통해 제공된 명령줄 인수 및 환경 변수에서 매개변수를 읽었으며, S3에서 다운로드하는 것도 SageMaker가 처리하기 때문에 로컬 폴더에서 데이터를 로드했습니다.

SageMaker API를 사용함으로써(단순히 노트북에서 로컬로 작업하는 대신) 실험의 추적성과 재현성을 향상시킬 수 있습니다. 또한 컴퓨팅 리소스 사용을 최적화하고 훈련된 모델에서 프로덕션 배포까지의 경로를 가속화할 수 있습니다.